<a href="https://colab.research.google.com/github/Frosti385/Programmierprojekt/blob/developLars/programmierprojekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Google Drive mounten und die Dateien als Cache reinladen (Bessere Laufzeit)

from google.colab import drive
drive.mount('/content/drive')
!mkdir "/content/drive2/"
%cp -av "/content/drive/MyDrive/data" "/content/drive2/"
%cp -av "/content/drive/MyDrive/garbage_files/" "/content/drive2/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive2/’: File exists
'/content/drive/MyDrive/data/plastic/plastic323.jpg' -> '/content/drive2/data/plastic/plastic323.jpg'
'/content/drive/MyDrive/data/plastic/plastic253.jpg' -> '/content/drive2/data/plastic/plastic253.jpg'
'/content/drive/MyDrive/data/plastic/plastic381.jpg' -> '/content/drive2/data/plastic/plastic381.jpg'
'/content/drive/MyDrive/data/plastic/plastic209.jpg' -> '/content/drive2/data/plastic/plastic209.jpg'
'/content/drive/MyDrive/data/plastic/plastic169.jpg' -> '/content/drive2/data/plastic/plastic169.jpg'
'/content/drive/MyDrive/data/plastic/plastic302.jpg' -> '/content/drive2/data/plastic/plastic302.jpg'
'/content/drive/MyDrive/data/plastic/plastic194.jpg' -> '/content/drive2/data/plastic/plastic194.jpg'
'/content/drive/MyDrive/data/plastic/plastic389.jpg' -> '/content/drive2/data/plastic/plastic3

In [ ]:
# Netz trainieren VGG16


import os
import torch
import torchvision
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import make_grid
import numpy as np


def load_data(path: str):
    with open(path, "r") as f:
        files = f.readlines()
        targets = []
        images = []

        for line in files:
            name, target = line.split(" ")
            if name.startswith("paper"):
                path = f"/content/drive2/data/paper/{name}"
            elif name.startswith("glass"):
                path = f"/content/drive2/data/glass/{name}"
            elif name.startswith("metal"):
                path = f"/content/drive2/data/metal/{name}"
            elif name.startswith("cardboard"):
                path = f"/content/drive2/data/cardboard/{name}"
            elif name.startswith("plastic"):
                path = f"/content/drive2/data/plastic/{name}"
            elif name.startswith("trash"):
                path = f"/content/drive2/data/trash/{name}"

            targets.append(int(target)-1)
            image = torchvision.io.read_image(path)
            images.append(image)

    return images, targets


class TrashDataset(Dataset):

  def __init__(self, images, targets):
    self.targets = targets
    self.images = images

  def __len__(self):
    return len(self.targets)

  transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((224, 224)),
     transforms.ToTensor()])

  def __getitem__(self, index):
    return self.targets[index], self.transform(self.images[index])


class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = torchvision.models.vgg16(pretrained=True)
    self.fc1 = nn.Linear(1000, 6)
  def forward(self, x):
    x = self.network(x)
    x = self.fc1(x)
    return x


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 40
cv_scores = []

train_images, train_targets = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_train.txt")
train_dataset = TrashDataset(images=train_images, targets=train_targets)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

cv_images, cv_targets = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_val.txt")
cv_dataset = TrashDataset(images=cv_images, targets=cv_targets)
cv_loader = torch.utils.data.DataLoader(cv_dataset, batch_size=batch_size, shuffle=True, num_workers=0)


net = Net()
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

for epoch in range(20):
    for idx, data in enumerate(train_loader):
        targets, inputs = data

        if torch.cuda.is_available():
          targets, inputs = targets.cuda(), inputs.cuda()

        optimizer_ft.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer_ft.step()
        print(f'[{epoch + 1}, {idx + 1:5d}] loss: {loss}')

        correct = 0
        total = 0
    with torch.no_grad():
        for data in cv_loader:
            targets, inputs = data

            if torch.cuda.is_available():
              targets, inputs = targets.cuda(), inputs.cuda()
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        print(f'Accuracy of the network on the cv images: {100 * correct // total} %')
        
        torch.save(net, "/content/drive/MyDrive/models/epoch.max")

        cv_scores.append(100 * correct // total)



    
print('Finished Training')
print(cv_scores)


In [7]:
# Netz trainieren Resnet


import os
import torch
import torchvision
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import make_grid
import numpy as np


def load_data(path: str):
    with open(path, "r") as f:
        files = f.readlines()
        targets = []
        images = []

        for line in files:
            name, target = line.split(" ")
            if name.startswith("paper"):
                path = f"/content/drive2/data/paper/{name}"
            elif name.startswith("glass"):
                path = f"/content/drive2/data/glass/{name}"
            elif name.startswith("metal"):
                path = f"/content/drive2/data/metal/{name}"
            elif name.startswith("cardboard"):
                path = f"/content/drive2/data/cardboard/{name}"
            elif name.startswith("plastic"):
                path = f"/content/drive2/data/plastic/{name}"
            elif name.startswith("trash"):
                path = f"/content/drive2/data/trash/{name}"

            targets.append(int(target)-1)
            image = torchvision.io.read_image(path)
            images.append(image)

    return images, targets


class TrashDataset(Dataset):

  def __init__(self, images, targets):
    self.targets = targets
    self.images = images

  def __len__(self):
    return len(self.targets)

  transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((224, 224)),
     transforms.ToTensor()])

  def __getitem__(self, index):
    return self.targets[index], self.transform(self.images[index])


class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = torchvision.models.resnet152(pretrained=True)
    self.fc1 = nn.Linear(1000, 6)
  def forward(self, x):
    x = self.network(x)
    x = self.fc1(x)
    return x


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 40
cv_scores = []

train_images, train_targets = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_train.txt")
train_dataset = TrashDataset(images=train_images, targets=train_targets)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

cv_images, cv_targets = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_val.txt")
cv_dataset = TrashDataset(images=cv_images, targets=cv_targets)
cv_loader = torch.utils.data.DataLoader(cv_dataset, batch_size=batch_size, shuffle=True, num_workers=0)


net = Net()
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

for epoch in range(20):
    for idx, data in enumerate(train_loader):
        targets, inputs = data

        if torch.cuda.is_available():
          targets, inputs = targets.cuda(), inputs.cuda()

        optimizer_ft.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer_ft.step()
        print(f'[{epoch + 1}, {idx + 1:5d}] loss: {loss}')

        correct = 0
        total = 0
    with torch.no_grad():
        for data in cv_loader:
            targets, inputs = data

            if torch.cuda.is_available():
              targets, inputs = targets.cuda(), inputs.cuda()
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        print(f'Accuracy of the network on the cv images: {100 * correct // total} %')
        
        torch.save(net, "/content/drive/MyDrive/models/epoch.max")

        cv_scores.append(100 * correct // total)



    
print('Finished Training')
print(cv_scores)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

[1,     1] loss: 2.0548012256622314
[1,     2] loss: 1.9073898792266846
[1,     3] loss: 1.9216699600219727
[1,     4] loss: 1.8416019678115845
[1,     5] loss: 1.6112163066864014
[1,     6] loss: 1.5134004354476929
[1,     7] loss: 1.1915030479431152
[1,     8] loss: 1.162987470626831
[1,     9] loss: 1.2451826333999634
[1,    10] loss: 0.9992696046829224
[1,    11] loss: 1.0410367250442505
[1,    12] loss: 1.1361973285675049
[1,    13] loss: 0.8695770502090454
[1,    14] loss: 0.9345672726631165
[1,    15] loss: 0.6157571077346802
[1,    16] loss: 0.8913310766220093
[1,    17] loss: 0.8012412786483765
[1,    18] loss: 0.6893352270126343
[1,    19] loss: 0.9737428426742554
[1,    20] loss: 0.7979055643081665
[1,    21] loss: 0.7048569917678833
[1,    22] loss: 0.5261493921279907
[1,    23] loss: 0.5293549299240112
[1,    24] loss: 0.7298299074172974
[1,    25] loss: 0.5982457995414734
[1,    26] loss: 0.5391678214073181
[1,    27] loss: 0.535876452922821
[1,    28] loss: 0.59658348560

In [ ]:
# Packages Laden, Funktionieren Definieren, Klassen erstellen - falls nur die Testdaten ausgeführt werden sollen

import os
import torch
import torchvision
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import make_grid
import numpy as np


def load_data(path: str):
    with open(path, "r") as f:
        files = f.readlines()
        targets = []
        images = []

        for line in files:
            name, target = line.split(" ")
            if name.startswith("paper"):
                path = f"/content/drive2/data/paper/{name}"
            elif name.startswith("glass"):
                path = f"/content/drive2/data/glass/{name}"
            elif name.startswith("metal"):
                path = f"/content/drive2/data/metal/{name}"
            elif name.startswith("cardboard"):
                path = f"/content/drive2/data/cardboard/{name}"
            elif name.startswith("plastic"):
                path = f"/content/drive2/data/plastic/{name}"
            elif name.startswith("trash"):
                path = f"/content/drive2/data/trash/{name}"

            targets.append(int(target)-1)
            image = torchvision.io.read_image(path)
            images.append(image)

    return images, targets



class TrashDataset(Dataset):

  def __init__(self, images, targets):
    self.targets = targets
    self.images = images

  def __len__(self):
    return len(self.targets)

  transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((224, 224)),
     transforms.ToTensor()])

  def __getitem__(self, index):
    return self.targets[index], self.transform(self.images[index])



In [8]:
# Testdaten laden und Netz evaluieren

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classes = ['Glas', 'Papier', 'Pappe', 'Plastik', 'Metall', 'Müll']
batch_size = 40

model = torch.load("/content/drive/MyDrive/models/epoch.max", map_location=torch.device(device))
try_image, try_target = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_test.txt")
try_dataset = TrashDataset(images=try_image, targets=try_target)
try_loader = torch.utils.data.DataLoader(try_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

"""
for data in try_loader:
  targets, inputs = data
  #targets, inputs = targets.cuda(), inputs.cuda()
  outputs = net(inputs)
  _, predicted = torch.max(outputs.data,1)
  print(f"Der Müll auf dem Bild ist {classes[predicted]}")
"""
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for data in try_loader:
    targets, inputs = data
    if torch.cuda.is_available():
      targets, inputs = targets.cuda(), inputs.cuda()
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()
  print(f'Accuracy of the network on the test images: {100 * correct // total} %')
        



Accuracy of the network on the test images: 92 %
